# Génération du dataset

In [1]:
from pulp import *
import numpy as np
import random
from dijkstar import Graph, find_path

# Creation d'une ville sur une carte donnée
def generate_nodes(number,max_x=1080,max_y=720):
    """
    Créer une liste de villes sous forme de coordonnées
    
    Parameters
    ----------
    number : int
        Nombre de ville a générer sur la carte
    max_x : int, optional
        Maximum de la largeur de la carte. The default is 1080.
    max_y : int, optional
        Maximum de la hauteur de la carte. The default is 720.
    """
    # Initialisation de la liste de villes
    nodes = []
    # Boucle pour générer les villes
    for i in range(number):
        # Génération des coordonnées de la ville
        nodes.append([random.randint(0.1*max_y,max_x-0.1*max_y),random.randint(0.1*max_y,max_y-0.1*max_y)])

    return nodes



# Retourne la distance entre 2 points (x1,y1) et (x2,y2)
def distance(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5



# Retourne la matrice de distance entre les villes
def coordinats_to_matrice(nodes,prob):
    """
    Créer une matrice de distance entre les villes
    
    Parameters
    ----------
    nodes : list
        Liste de villes
    """
    matrice = np.ones((len(nodes),len(nodes)))*np.inf
    for x in range(len(nodes)):
        for y in range(len(nodes)):
            if random.uniform(0,1)<prob:
                matrice[x][y] = distance(nodes[x][0],nodes[x][1],nodes[y][0],nodes[y][1])
                matrice[y][x] = matrice[x][y]
            if x == y :
                matrice[x][x] = np.inf
    return matrice



# Génère une liste de villes par lesquels passer 
def random_city(graph, nb_city):
    if (nb_city <= len(graph)):
        return random.sample(range(len(graph)),nb_city)



# Convertit le graphe en format de dijkstra
def convert_graph(graph):
    graph_dijkstra = Graph()
    for i in range(len(graph)):
        for j in range(len(graph)):
            graph_dijkstra.add_edge(i, j,graph[i][j])
    return graph_dijkstra



# Génère le graphe complet des villes séléctionnées
def cities_complet(graph, cities, format_is_inf):
    """
    Créer un graphe complet des villes séléctionnées

    Parameters
    ----------
    graph : list
        Graphe des distances entre les villes
    cities : list
        Liste de villes séléctionnées
    """
    graph_cities = np.zeros((len(cities),len(cities)))
    graph_dji = convert_graph(graph)
    for index_city, city in enumerate(cities):
        for index_city2, city2 in enumerate(cities):
            graph_cities[index_city][index_city2] = graph[city][city2]
    for index, city in enumerate(graph_cities):
        liaisons = np.where(city == np.inf)
        liaisons = np.delete(liaisons,np.where(liaisons[0] == index))
        for col in liaisons:
            shortest_path = find_path(graph_dji,cities[index],cities[col])
            graph_cities[index][col] = shortest_path.total_cost
    if format_is_inf == False:
        for i in range(len(cities)):
            graph_cities[i][i] = 0
    return graph_cities




def input_output_contrainte(matrice, max_value):
    for i,line in enumerate(matrice):
        arr = np.where(line == np.inf)[0]
        arr = np.delete(arr,np.where(arr == i))
        while len(arr) > len(matrice)-3:
            index = random.choice(arr)
            matrice[i][index] = random.randint(1,max_value)
            arr = np.delete(arr,np.where(arr == index))
    for i in range(len(matrice)):
        for j in range(len(matrice)):
            if matrice[i][j] != np.inf:
                matrice[j][i] = matrice[i][j]
    return matrice



# Génère toutes le dataset d'une instance
def random_matrix_generator(size_graph,size_cities, max_value_x, max_value_y, prob, format_is_inf=True):
    """
    Génère toutes les matrices d'une instance

    Parameters
    ----------
    size_graph : int
        Taille du graphe
    size_cities : int
        Taille des villes
    max_value : int
        Valeur maximale des distances
    prob : float
        Probabilité de la distance infini
    complet_is_inf : bool, optional
        True si le format de sortie du graphe complet doit être infini. The default is False.
    """
    # Génération des villes
    matrix_coordonate = generate_nodes(size_graph,max_value_x,max_value_y)
    # Génération de la matrice de distance
    matrix_distance = coordinats_to_matrice(matrix_coordonate,prob)
    # Génération si full 0 sur la ligne
    matrix_distance = input_output_contrainte(matrix_distance, max_value_x)
    # Création de la liste des villes
    random_cities = random_city(matrix_distance,size_cities)
    # Génération du graphe complet
    city_completed = cities_complet(matrix_distance, random_cities, format_is_inf)

    return matrix_coordonate, matrix_distance, random_cities, city_completed

In [2]:
nb_city_total = 50
nb_city_selectionned = 10
max_value_x = 1080
max_value_y = 720
probability = 0.2

matrix_coordonate, matrix_distance, random_cities, matrix_city_completed = random_matrix_generator(nb_city_total,nb_city_selectionned,max_value_x,max_value_y,probability,False)


print("Matrice coordonées : \n",matrix_coordonate)
print("\n---------------------------------------------------------------------------------------\n")
print("Matrice distance : \n",matrix_distance)
print("\n---------------------------------------------------------------------------------------\n")
print("Liste des villes : \n",random_cities)
print("\n---------------------------------------------------------------------------------------\n")
print("Graphe complet : \n",matrix_city_completed)


Matrice coordonées : 
 [[274, 214], [553, 475], [683, 210], [74, 403], [942, 472], [956, 473], [504, 136], [579, 339], [427, 448], [873, 399], [292, 296], [800, 153], [373, 401], [100, 139], [1001, 86], [635, 301], [1004, 165], [598, 85], [373, 304], [229, 216], [555, 639], [261, 433], [335, 613], [113, 574], [915, 167], [197, 220], [233, 148], [428, 448], [383, 430], [436, 161], [159, 168], [226, 373], [843, 236], [720, 126], [981, 285], [725, 281], [751, 480], [476, 336], [592, 148], [771, 177], [101, 505], [292, 471], [365, 552], [354, 255], [474, 266], [150, 457], [540, 307], [279, 538], [711, 642], [971, 592]]

---------------------------------------------------------------------------------------

Matrice distance : 
 [[         inf 382.04973498          inf ...          inf          inf
           inf]
 [382.04973498          inf 295.16944286 ...          inf          inf
           inf]
 [         inf 295.16944286          inf ...          inf 432.90645641
  478.40150501]
 ...


# Algorithme linéaire

In [5]:
import pulp
import pandas as pd
from scipy.spatial import distance_matrix
from matplotlib import pyplot as plt
import time
import copy

#This function takes locations as input and plot a scatter plot
def plot_fig(loc,heading="plot"):
    plt.figure(figsize=(10,10))
    for i,row in loc.iterrows():
        if i==0:
            plt.scatter(row["x"],row["y"],c='r')
            plt.text(row["x"]+0.2, row["y"]+0.2, 'DELHI (depot) ')
        else:
            plt.scatter(row["x"], row["y"], c='black')
            plt.text(row["x"] + 0.2, row["y"] + 0.2,full_data.loc[i]['CITy'] )
        plt.ylim(6,36)
        plt.xlim(66,96)
        plt.title(heading)
# this function find all the subtour in the LP solution.
def get_plan(r0):
    r=copy.copy(r0)
    route = []
    while len(r) != 0:
        plan = [r[0]]
        del (r[0])
        l = 0
        while len(plan) > l:
            l = len(plan)
            for i, j in enumerate(r):
                if plan[-1][1] == j[0]:
                    plan.append(j)
                    del (r[i])
        route.append(plan)
    return(route)

def prog_lineaire(graph_complet_cities, random_cities):
    """_summary_

    Args:
        graphe : graphe complet de liste   
        random_cities : liste de villes séléctionnées

    Returns:
        route_plan: liste de la route
        subtour: liste des subtours
        time_taken: temps de calcul
        status : Statut de l'opti
        value : Valeur de l'objectif
    """
    no_of_locs = len(graph_complet_cities)

    dis_mat= graph_complet_cities

    start_t_1=time.time()
    model=pulp.LpProblem('tsp',pulp.LpMinimize)
    #define variable
    x=pulp.LpVariable.dicts("x",((i,j) for i in range(no_of_locs) \
                                    for j in range(no_of_locs)),\
                            cat='Binary')
    #set objective
    model+=pulp.lpSum(dis_mat[i][j]* x[i,j] for i in range(no_of_locs) \
                        for j in range(no_of_locs))
    # st constraints
    for i in range(no_of_locs):
        model+=x[i,i]==0
        model+=pulp.lpSum(x[i,j] for j in range(no_of_locs))==1
        model += pulp.lpSum(x[j, i] for j in range(no_of_locs)) == 1
        
    status=model.solve()

    route=[(i,j) for i in range(no_of_locs) \
            for j in range(no_of_locs) if pulp.value(x[i,j])==1]
    route_plan=get_plan(route)
    subtour=[]

    while len(route_plan)!=1:
        for i in range(len(route_plan)):
            model+=pulp.lpSum(x[route_plan[i][j][0],route_plan[i][j][1]]\
                                for j in range(len(route_plan[i])))<=\
                                len(route_plan[i])-1
        status=model.solve()
        route = [(i, j) for i in range(no_of_locs) \
                    for j in range(no_of_locs) if pulp.value(x[i, j]) == 1]
        route_plan = get_plan(route)
        
        subtour.append(len(route_plan))

    for i in range(len(route_plan[0])):
        temp_list = list(route_plan[0][i])
        temp_list[0], temp_list[1] = random_cities[route_plan[0][i][0]], random_cities[route_plan[0][i][1]]
        route_plan[0][i] = tuple(temp_list)

    return route_plan,subtour,time.time()-start_t_1,pulp.LpStatus[status],pulp.value(model.objective)

In [6]:
route_plan,subtour,time,status,value = prog_lineaire(matrix_city_completed, random_cities)

print("Route :", route_plan)
print("Subtour :", subtour)
print("Time :", time)
print("Status :", status)
print("Value :", value)

Route : [[(49, 32), (32, 5), (5, 10), (10, 47), (47, 16), (16, 36), (36, 21), (21, 2), (2, 1), (1, 49)]]
Subtour : [3, 3, 1]
Time : 0.21767735481262207
Status : Optimal
Value : 2682.660729323845


# Visualisation Pygame

In [1]:
import pygame

def draw_nodes_and_bases_routes(nodes, window, route_list):
    # draw nodes and bases routes
    #window.fill((255, 255, 255))
    """    for nodex in nodes:
        
        for nodey in nodes:
            # draw line from nodex to nodey
            pygame.draw.line(window, (100, 100, 100), nodex, nodey, 2)
            """

    for nodex in nodes:
        pygame.draw.circle(window, "#0D5C63", nodex, 20)

    for index in range(len(nodes)):
        font = pygame.font.SysFont('Comic Sans MS', 30)
        text_surface = font.render(str(index), True, (255, 255, 255))
        text_rect = text_surface.get_rect()
        text_rect.center = (nodes[index][0] , nodes[index][1])
        window.blit(text_surface, text_rect)

    for i in range(len(route_list[0])):
        pygame.draw.line(window, (100, 100, 100), nodes[route_list[0][i][0]], nodes[route_list[0][i][1]], 2)

    pygame.display.update()

    while True:
        if pygame.event.get(pygame.QUIT):
            break

    
    pygame.quit()

ModuleNotFoundError: No module named 'pygame'

In [8]:
import pygame

pygame.init()
window = pygame.display.set_mode((1080, 720))
window.fill((255, 255, 255))

draw_nodes_and_bases_routes(matrix_coordonate, window, route_plan)